# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る


#### **このファイルのミッション**
classを作る→最低限の入力で計算ができる関数を作る


#### **データ、条件**
small, binary data

### **調整したこと**
- XはSNPを想定。SNPの発生する箇所は95%ほとんどある塩基、5%ある別の塩基としか出現しないので0, 1で置換可能。
- Xは標準化しない
- 特徴量選択は内積が大きいものを取り出す
- ay-byは内積のままでok
- 場合によってはイジングモデルを採用しよう

## class
- 未完成

In [27]:
	# -*- coding: UTF-8 -*-
class Num :
    name = "数値"
    def __init__(self, v) :
        self.v = v
        
    def add(self, v) :
        x = Num(self.v + v)
        return x
    
    # クラスメソッド
    @classmethod
    def class_method(cls) :
        print(cls.name)
        
    # スタティックメソッド
    @staticmethod
    def static_method() :
        print(Num.name)
        
class Int(Num) :
    name = "整数"
    
class Complex(Num) :
    name = "複素数"
    def __init__(self, re, im = 0.0) :
        self.re = re
        self.im = im
        
    def add(self, v) :
        x = Complex(self.re + v.re, self.im + v.im)
        return x
    
# -*- coding: UTF-8 -*-
x = Complex(1, 2)
Complex.class_method()
x.class_method()
Complex.static_method()
x.static_method()

複素数
複素数
数値
数値


In [1]:
class Spam:
    def __init__(self,ham,egg):
        self.ham = ham
        self.egg = egg
    def output(self):
        sum = self.ham + self.egg
        print("{0}".format(sum))

spam = Spam(5,10)
spam.output()

15


In [23]:
class Hamil_sum0:
    def __init__(self,X,y, sel_col_num):
        self.X = X
        self.y = y
        self.sel_col_num = sel_col_num
        
    def drop_str0_X(self, X):
        X = self.X
        for each_col in X.columns:
            if X[each_col].std()==0.0:
                X = X.drop(each_col, axis=1)
        return X

In [24]:
Hamil_sum0 = Hamil_sum0(X_ori55, ori_y5, sel_col_num2)
Hamil_sum0

In [13]:
class Hamil_sum:
    def __init__(self,X,y, sel_col_num):
        self.X = X
        self.y = y
        self.sel_col_num = sel_col_num
        
    def drop_str0_X(self, X):
        X = self.X
        for each_col in X.columns:
            if X[each_col].std()==0.0:
                X = X.drop(each_col, axis=1)
        return X
    
    def featrure_InnerProduct_sel_mul(self, X, y, sel_col_num):
        X = self.X
        y = self.y
        sel_col_num = self.sel_col_num
        inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
        sort_inner_product_list = sorted(inner_product_list ,reverse=True)

        elect_cols = []
        for i in range(sel_col_num):
            iindex = inner_product_list.index(sort_inner_product_list[i])
            select_cols.append(X.columns[iindex])

        else_cols = list(X.columns)
        for col in select_cols:
            else_cols.remove(col)

        return select_cols, else_cols
    
    def make_ax_by(self, X, y, select_cols, else_cols):
        ij = []
        ax_by = []
        for i in select_cols:
            for j in else_cols:
                row_each_axby = np.dot(X[i], y) - np.dot(X[j], y)
                ax_by.append(row_each_axby)
        return ax_by
    
    def H(self, ax_by, index_ax_by):
        W = X.max(axis=1).max() *1*X.shape[0] - X.min(axis=1).min()*1*X.shape[0]
        z = [0]*W

        each_axby = ax_by[index_ax_by]
        sum_w_z = 0

        if each_axby >= 1:
            z[each_axby] = 1
            sum_w_z = sum([each_axby*z[w] for w in range(W)])
        else:
            z[0] = 1
            sum_w_z = sum([1*z[w] for w in range(W)])

        sum_z = sum(z)

        ans_h =  (1-sum_z)**2 + (sum_w_z - each_axby)**2
        return ans_h
    
    def H_sum(self, ax_by):
        H_sum =  sum([H(ax_by, index_ax_by) for index_ax_by in range(len(ax_by))])
        return H_sum

In [16]:
Hamil_sum = Hamil_sum(X_ori55, ori_y5, sel_col_num2)
H_sum = Hamil_sum.H_sum
H_sum

TypeError: 'Hamil_sum' object is not callable

##### import

In [28]:
import itertools
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random
from pyqubo import Array, OneHotEncInteger, solve_qubo
import scipy.stats

#### X

In [29]:
#標準偏差が0である列をなくす
def drop_str0_X(X):
    for each_col in X.columns:
        if X[each_col].std()==0.0:
            X = X.drop(each_col, axis=1)
    return X

In [32]:
X_ori55 = pd.read_csv("../input/SNP_df55.csv", sep=',', index_col=0)
X_ori55_ad = drop_str0_X(X_ori55)

In [33]:
max_X_element = X_ori55_ad.max(axis=1).max() 
min_X_element = X_ori55_ad.min(axis=1).min() 

num_samples = X_ori55_ad.shape[0]
num_cols = X_ori55_ad.shape[1]

#### y

In [34]:
ori_y5 = pd.Series([1, 0, 1, 1, 0])
#ori_y5

In [35]:
sel_col_num1 = 1
sel_col_num2 = 2
sel_col_num3 = 3

#### 特徴量選択

In [60]:
def featrure_InnerProduct_sel_mul(X, y, sel_col_num):
    inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
    sort_inner_product_list = sorted(inner_product_list ,reverse=True)
    
    if sort_inner_product_list[sel_col_num-1] == sort_inner_product_list[sel_col_num]:
        return 'error', 'please change the number'
    else:  
        select_cols = []
        for i in range(sel_col_num):
            iindex = inner_product_list.index(sort_inner_product_list[i])
            select_cols.append(X.columns[iindex])

        else_cols = list(X.columns)
        for col in select_cols:
            else_cols.remove(col)

        return select_cols, else_cols

In [62]:
select_cols, else_cols = featrure_InnerProduct_sel_mul(X_ori55_ad, ori_y5, selected_col_num1)
select_cols

['B']

### ハミルトニアン
したいこと→ハミルトニアンを最小化させる（xを最適化）

In [63]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = max_X_element *1*num_samples - min_X_element*1*num_samples

In [64]:
# while True & if break バージョン
def make_y(y_after):
    while True:
        select_index = random.randrange(num_samples)
        y_after[select_index] = 1 - y_after[select_index]
        if np.dot(np.array([1]*num_samples).T, y_after) == sum(list(ori_y)):
            if y_after.std():
                #print(y_after.std())
                break
    return y_after 

In [65]:
def make_ax_by(X, y, select_cols, else_cols):
    ij = []
    ax_by = []
    for i in select_cols:
        for j in else_cols:
            row_each_axby = np.dot(X[i], y) - np.dot(X[j], y)
            ij.append([i, j])
            ax_by.append(row_each_axby)
    return ij, ax_by

In [73]:
def H(ax_by, index_ax_by):
    # W : ay - byの最大値
    z = [0]*W
        
    each_axby = ax_by[index_ax_by]
    sum_w_z = 0
    
    if each_axby >= 1:
        z[each_axby] = 1
        sum_w_z = sum([each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z - each_axby)**2
    return ans_h

In [74]:
def H_sum(ax_by):
    H_sum =  sum([H(ax_by, index_ax_by) for index_ax_by in range(len(ax_by))])
    return H_sum

In [75]:
ij, ax_by = make_ax_by(X_ori55_ad, ori_y5, select_cols, else_cols)

In [76]:
ij

[['B', 'A'], ['B', 'C'], ['B', 'D'], ['B', 'E']]

In [77]:
ax_by

[1, 1, 1, 1]

In [78]:
ori_H_sum = H_sum(ax_by)
ori_H_sum

0